# Week 0: Setup Verification

Welcome to the **Coding with AI** seminar! 

This notebook helps you verify that your environment is correctly configured before we start Day 1. It checks:
1.  **Google Cloud Project Connection** (Billing & API enablement)
2.  **library Installation** (`google-genai`)
3.  **Google Drive Integration** (Saving files)

## Step 1: Install Libraries & Authenticate

Run the cell below. It will:
1.  Install the necessary Python libraries.
2.  Ask you to authenticate with Google (follow the pop-up).
3.  Ask for your **Google Cloud Project ID** (from Part 6 of the Setup Guide).
    *   *Note: This is the ID (e.g., `my-project-123`), not just the name.*

In [ ]:
import os
import sys
from pathlib import Path

# Check if running in Google Colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    print("Running in Google Colab. Installing dependencies...")
    !pip install -q google-genai google-auth python-dotenv numpy
    
    from google.colab import auth
    print("Authenticating user...")
    auth.authenticate_user()
    
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
        
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
        print(f"Project ID set to: {PROJECT_ID}")
    else:
        print("Using default project from Application Default Credentials")
else:
    # Local setup handling (for instructors or local dev)
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass

## Step 2: Initialize Vertex AI Client

This step connects to the Google Cloud Vertex AI service using your credentials. If you see a `PermissionDenied` error here, it usually means:
1.  You didn't enable the **Vertex AI API** in your project.
2.  You entered the wrong **Project ID**.

In [ ]:
import google.auth
from google import genai

try:
    creds, project = google.auth.default()
    project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
    
    client = genai.Client(vertexai=True, project=project, location="us-central1")
    print(f"✓ Client initialized successfully using project: {project}")
except Exception as e:
    print(f"❌ ERROR: Could not initialize client. Details: {e}")

## Step 3: Test Gemini API Call

Now we'll make a real call to the model. This verifies your educational credits are active and working.

In [ ]:
try:
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents="Explain in one sentence why verifying your coding environment is important."
    )
    print("\n🤖 Model Response:")
    print(f"> {response.text}")
    print("\n✓ SUCCESS: API communication working!")
except Exception as e:
    print(f"❌ ERROR: API call failed. {e}")
    print("Tip: Check if you have redeemd your credits and linked them to this project!")

## Step 4: Test Google Drive Connection

Finally, let's verify you can save files to your Google Drive. This is where we will store your notebook saves.

In [ ]:
from google.colab import drive

if IN_COLAB:
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
    
    test_path = '/content/drive/My Drive/cwru-setup-test.txt'
    try:
        with open(test_path, 'w') as f:
            f.write('Setup Verification Complete!')
        print(f"✓ SUCCESS: Successfully wrote to {test_path}")
        print("Check your Google Drive for this file!")
    except Exception as e:
        print(f"❌ ERROR: Could not write to Drive. {e}")
else:
    print("Skipping Drive mount (running locally)")

## 🎉 You are Ready!

If you see three green checkmarks (Client init, Model response, Drive write), your environment is 100% ready for the course.

See you in class!